# FGSM/PGD epsilon sweep

Run FGSM and PGD for multiple models and epsilons, save outputs, and plot adversarial accuracy.

In [ ]:
import os
import re
import subprocess
import sys
from pathlib import Path

import matplotlib.pyplot as plt

EPSILONS = [0, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3]
MODELS = ["resnet18", "resnet34", "densenet121"]
DATASET = "cifar10"
MAX_IMAGES = 50
FGSM_SCRIPT = str(Path("fgsm.py").resolve())
PGD_SCRIPT = str(Path("pgd.py").resolve())
OUTPUT_ROOT = Path("output")

def alpha_for_epsilon(epsilon: float) -> float:
    return epsilon * (0.007 / 0.031)


In [ ]:
def run_fgsm(model: str, epsilon: float) -> float:
    eps_str = str(epsilon)
    save_dir = OUTPUT_ROOT / f"{DATASET}_{model}_{eps_str}"
    cmd = [
        sys.executable,
        FGSM_SCRIPT,
        "--model",
        model,
        "--dataset",
        DATASET,
        "--epsilon",
        eps_str,
        "--max-images",
        str(MAX_IMAGES),
        "--save-dir",
        str(save_dir),
    ]
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    match = re.search(r"FGSM acc: ([0-9.]+)", result.stdout)
    if not match:
        raise RuntimeError(f"Could not parse FGSM acc for {model} eps={epsilon}.")
    return float(match.group(1))


In [ ]:
def run_pgd(model: str, epsilon: float) -> float:
    eps_str = str(epsilon)
    alpha = alpha_for_epsilon(epsilon)
    save_dir = OUTPUT_ROOT / "pgd" / f"{DATASET}_{model}_{eps_str}"
    cmd = [
        sys.executable,
        PGD_SCRIPT,
        "--model",
        model,
        "--dataset",
        DATASET,
        "--epsilon",
        eps_str,
        "--alpha",
        str(alpha),
        "--max-images",
        str(MAX_IMAGES),
        "--save-dir",
        str(save_dir),
    ]
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    match = re.search(r"PGD acc: ([0-9.]+)", result.stdout)
    if not match:
        raise RuntimeError(f"Could not parse PGD acc for {model} eps={epsilon}.")
    return float(match.group(1))


In [ ]:
# FGSM sweep
accuracies = {model: [] for model in MODELS}

for model in MODELS:
    for epsilon in EPSILONS:
        acc = run_fgsm(model, epsilon)
        accuracies[model].append(acc)

accuracies


In [ ]:
# PGD sweep
pgd_accuracies = {model: [] for model in MODELS}

for model in MODELS:
    for epsilon in EPSILONS:
        acc = run_pgd(model, epsilon)
        pgd_accuracies[model].append(acc)

pgd_accuracies


In [ ]:
# Save FGSM CSV
import csv

fgsm_csv = Path("output") / "fgsm" / "fgsm_accuracy.csv"
fgsm_csv.parent.mkdir(parents=True, exist_ok=True)

with fgsm_csv.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["model", "epsilon", "accuracy"])
    for model in MODELS:
        for epsilon, acc in zip(EPSILONS, accuracies[model]):
            writer.writerow([model, epsilon, acc])

fgsm_csv


In [ ]:
# Save PGD CSV
import csv

pgd_csv = Path("output") / "pgd" / "pgd_accuracy.csv"
pgd_csv.parent.mkdir(parents=True, exist_ok=True)

with pgd_csv.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["model", "epsilon", "alpha", "accuracy"])
    for model in MODELS:
        for epsilon, acc in zip(EPSILONS, pgd_accuracies[model]):
            writer.writerow([model, epsilon, alpha_for_epsilon(epsilon), acc])

pgd_csv


In [ ]:
# Plot FGSM
plt.figure(figsize=(8, 5))
for model in MODELS:
    plt.plot(EPSILONS, accuracies[model], marker="o", label=model)

plt.xlabel("epsilon")
plt.ylabel("accuracy")
plt.title("FGSM adversarial accuracy vs. epsilon")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Plot PGD
plt.figure(figsize=(8, 5))
for model in MODELS:
    plt.plot(EPSILONS, pgd_accuracies[model], marker="o", label=model)

plt.xlabel("epsilon")
plt.ylabel("accuracy")
plt.title("PGD adversarial accuracy vs. epsilon")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
